In [ ]:
## In Class 2021-09-13

In [3]:
## Q1 Load the data file to you S3 bucket. Using the pandas library, read the csv data file and
## create a data-frame called car_price.

import pandas as pd
import boto3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV

## Defining the S3 bucket
s3 = boto3.resource('s3')
bucket_name = 'bonnieh-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'CarPrice_Assignment.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
car_price = pd.read_csv(file_content_stream)
car_price.head()

car_ID  symboling                   CarName fueltype aspiration doornumber  \
0       1          3        alfa-romero giulia      gas        std        two   
1       2          3       alfa-romero stelvio      gas        std        two   
2       3          1  alfa-romero Quadrifoglio      gas        std        two   
3       4          2               audi 100 ls      gas        std       four   
4       5          2                audi 100ls      gas        std       four   

       carbody drivewheel enginelocation  wheelbase  ...  enginesize  \
0  convertible        rwd          front       88.6  ...         130   
1  convertible        rwd          front       88.6  ...         130   
2    hatchback        rwd          front       94.5  ...         152   
3        sedan        fwd          front       99.8  ...         109   
4        sedan        4wd          front       99.4  ...         136   

   fuelsystem  boreratio  stroke compressionratio horsepower  peakrpm citympg  \
0        mpfi       3.47    2.68              9.0        111     5000      21   
1        mpfi       3.47    2.68              9.0        111     5000      21   
2        mpfi       2.68    3.47              9.0        154     5000      19   
3        mpfi       3.19    3.40             10.0        102     5500      24   
4        mpfi       3.19    3.40              8.0        115     5500      18   

   highwaympg    price  
0          27  13495.0  
1          27  16500.0  
2          26  16500.0  
3          30  13950.0  
4          22  17450.0  

[5 rows x 26 columns]

In [15]:
## Q2 Using the wheelbase, enginesize, compressionratio, horsepower, peakrpm, citympg,
## and highwaympg as the predictor variables, and price is the target variable, split the data into train
## (80%) and test (20%).

X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg',
'highwaympg']]
Y = car_price['price']

## splitting the data

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

In [16]:
## Using the train dataset, perform LASSO as a variable selector.

## estimating lambda (alpha)--do this first
## don't apply on entire data set--or get data leakage--only do on training data

lasso_cv = LassoCV(alphas = [0.001, 0.01, 0.1, 1, 10], normalize = True, cv = 5).fit(X_train, Y_train)

## extracting optimal alpha

cv_alpha = lasso_cv.alpha_
cv_alpha

10.0

In [18]:
## Variable selection with LASSO

lasso_md = Lasso(alpha = cv_alpha, normalize = True).fit(X_train, Y_train)
lasso_md.coef_

## notice the zero--which means last variable here (highway mpg) 

array([ 150.5946608 ,  103.71425291,  261.84675118,   51.67291654,
          1.6742568 , -125.253109  ,   -0.        ])

In [19]:
## Q4 Using the variables from LASSO regression, normalize the inputs variables of the train and
## test datasets using the L2 normalization. That is, for each input variable subtract the mean of that
## variable, then divide by the L2-norm of that variable.

## dropping highway
## axis = 1 is drop column, axis = 0 is drop row

X_train = X_train.drop(columns = ['highwaympg'], axis = 1)
X_test = X_test.drop(columns = ['highwaympg'], axis = 1)

In [23]:
def l2_normalization(X):
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(x**2))
    return (X-x_mean) /l2


In [24]:
## normalizing the trzin and test

X_train = X_train.apply(l2_normalization,axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

NameError: name 'x' is not defined

In [ ]:
##. building the linear regression model
lm_md = LinearREgression().fit(X_train, Y_train)

